In [4]:
%%html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>3D Talking Cat Mock-up</title>
    <!-- Load Tailwind CSS for styling -->
    <script src="https://cdn.tailwindcss.com"></script>
    <style>
        /* Custom CSS for 3D canvas and layout */
        body {
            font-family: 'Inter', sans-serif;
            margin: 0;
            overflow: hidden;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            min-height: 100vh;
            background-color: #1e293b; /* Slate-800 background */
            color: #f1f5f9; /* Slate-100 text */
            transition: background-color 0.3s;
        }
        #container {
            width: 100%;
            max-width: 800px;
            height: 60vh;
            min-height: 400px;
            position: relative;
            background-color: #0f172a; /* Slate-900 for the visual frame */
            border-radius: 1.5rem;
            box-shadow: 0 25px 50px -12px rgba(0, 0, 0, 0.5);
            margin-bottom: 1rem;
            overflow: hidden; /* Ensure canvas stays within bounds */
        }
        canvas {
            display: block;
            width: 100%;
            height: 100%;
        }
        .controls-group {
            display: flex;
            gap: 1rem;
            margin-top: 1rem;
        }
        /* Modal styling */
        .modal {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background-color: rgba(0, 0, 0, 0.7);
            display: none;
            justify-content: center;
            align-items: center;
            z-index: 100;
        }
        .modal-content {
            background-color: #2d3748; /* Dark background */
            padding: 2rem;
            border-radius: 0.75rem;
            max-width: 400px;
            box-shadow: 0 10px 20px rgba(0, 0, 0, 0.5);
        }
    </style>
    <!-- Load Three.js (3D Library) and Tone.js (Audio Synthesis) -->
    <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
    <script src="https://unpkg.com/tone@14.9.11/build/Tone.js"></script>
</head>
<body>

    <div class="controls mb-4 p-4 bg-gray-700/50 rounded-xl shadow-lg flex flex-col items-center">
        <h1 class="text-3xl font-bold mb-2 text-yellow-300">Whiskerbot 3000 (3D)</h1>
        <p id="statusMessage" class="text-sm text-gray-300 mb-4 transition-all duration-300">
            Click 'Talk' to activate the lip-sync and sound.
        </p>
        <div class="controls-group">
            <button id="talkButton" class="px-6 py-3 bg-indigo-500 hover:bg-indigo-600 text-white font-semibold rounded-full shadow-xl transition transform hover:scale-105 active:bg-indigo-700">
                Make Cat Talk!
            </button>
            <button id="exportButton" class="px-6 py-3 bg-green-500 hover:bg-green-600 text-white font-semibold rounded-full shadow-xl transition transform hover:scale-105 active:bg-green-700">
                Export/Save File
            </button>
        </div>
    </div>

    <div id="container">
        <!-- Three.js renderer will append the canvas here -->
    </div>

    <!-- Export Modal -->
    <div id="exportModal" class="modal">
        <div class="modal-content">
            <h2 class="text-xl font-bold mb-4 text-green-400">Export File Instructions</h2>
            <p class="text-gray-300 mb-4">
                Since this application is a single HTML file running entirely in your browser,
                you can save (export) the complete file using your browser's function:
            </p>
            <ol class="list-decimal list-inside space-y-2 text-left text-gray-200">
                <li>**Right-Click** anywhere on the page (outside the cat/controls).</li>
                <li>Select **'Save Page As...'** or **'Save As...'**</li>
                <li>Ensure the file type is set to **'Webpage, Complete'** or **'HTML Only'**.</li>
                <li>Name the file (e.g., `talking_cat.html`) and click Save.</li>
            </ol>
            <button onclick="document.getElementById('exportModal').style.display='none';"
                    class="mt-6 px-4 py-2 bg-red-500 hover:bg-red-600 rounded-lg text-white font-semibold transition">
                Close
            </button>
        </div>
    </div>

    <script>
        // --- Configuration and Initialization ---
        let scene, camera, renderer, catGroup;
        let mouthMesh;
        let isTalking = false;

        const container = document.getElementById('container');
        const talkButton = document.getElementById('talkButton');
        const exportButton = document.getElementById('exportButton');
        const statusMessage = document.getElementById('statusMessage');

        // Initialize Tone.js for synthesized speech sound
        const synth = new Tone.Synth({
            oscillator: { type: "triangle" },
            envelope: { attack: 0.005, decay: 0.1, sustain: 0.3, release: 1 }
        }).toDestination();

        // ----------------------------------------------------
        // --- Three.js Scene Setup ---
        // ----------------------------------------------------

        function init() {
            // 1. Scene
            scene = new THREE.Scene();
            scene.background = new THREE.Color(0x0f172a);

            // 2. Camera
